#### bibliotecas necessárias
- pandas==2.2.2
- llama-index==0.11.20
- llama-index-llms-groq==0.2.0
- llama-index-experimental==0.4.0
- gradio==5.4.0
- fpdf==1.7.2

pip install llama-index==0.11.20 && pip install llama-index-llms-groq==0.2.0 && pip install llama-index-experimental==0.4.0 && pip install gradio==5.4.0 && pip install fpdf==1.7.2

- OBS: Também é necessário uma key atravez do https://groq.com, sinalizado na variável "groq_key" na celula abaixo.
- OBS2: Bem como um arquivo para análise, no caso, para este projeto utilizamos as informações de uma tabela fictícia da Valquíria Alencar, instrutora do curso de LLM LlamaIndex na Alura

In [ ]:
import pandas as pd
import textwrap
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.experimental.query_engine import PandasQueryEngine
groq_key = ''
url = 'https://github.com/alura-cursos/llamaIndex_pandas_query/blob/main/Dados/vendas.csv'

Settings.llm = Groq(model='llama3-70b-8192', api_key=groq_key)

In [ ]:
DF = pd.read_csv(url)

In [ ]:
# A FORMA MAIS DIRETA DE RESPOSTAS UTILIZANDO QUERY_ENGINE
query_engine = PandasQueryEngine(df=DF, verbose=True)

response = query_engine.query('Qual a forma de pagamento mais utilizada?')

In [ ]:
# RESPOSTAS MAIS COMPLETAS ALÉM DO VALOR FINAL, UMA EXPLICAÇÃO
# TEXTWRAP É APENAS PARA FACILITAR A VISUALIZAÇÃO, NÃO É NECESSÁRIO
query_engine = PandasQueryEngine(df=DF, verbose=True, synthesize_response=True)

response = query_engine.query('Qual a média de avaliação para cada filial?')
print(textwrap.fill(response.response, width=100))

In [ ]:
# É POSSIVEL FAZER PLOTAGEM DE GRAFICOS, UMA VEZ QUE O PANDAS POSSUI O PLOT
query_engine = PandasQueryEngine(df=DF, verbose=True, synthesize_response=True)

response = query_engine.query('Você pode plotar a distribuição das avaliações?')

Quando pedimos uma ação do modelo que não seja uma pergunta, como se fosse uma ordem, o modelo pode responder sem a formatação de linguagem (portugues), e nos trás uma resposta em inglês. Na maioria dos cenários isto é um problema, portanto, vamos fazer os ajustes necessários no prompt

query_engine = PandasQueryEngine(df=DF, verbose=True, synthesize_response=True)

response = query_engine.query('Plote a distribuição das avaliações')

In [ ]:
from llama_index.core import PromptTemplate
from llama_index.experimental.query_engine.pandas import PandasInstructionParser

In [ ]:
# FUNÇÃO PARA OBTER UMA DESCRIÇÃO DAS COLUNAS DO DATAFRAME
def descricao_colunas(DF):
    descricao = '\n'.join([f"`{col}`: {str(DF[col].dtype)}" for col in DF.columns])
    return 'Aqui estão os detalhes das colunas do DataFrame:\n' + descricao

# INSTRUÇÕES PARA ORIENTAR O MODELO A CONVERTER UMA CONSULTA EM LINGUAGEM NATURAL EM CÓDIGO PYTHON EXECUTÁVEL COM A BIBLIOTECA PANDAS
instruction_str = ("1. Converta a consulta para código Python executável usando Pandas.\n"
                    "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
                    "3. O código deve representar uma solução para a consulta.\n"
                    "4. IMPRIMA APENAS A EXPRESSÃO.\n"
                    "5. Não coloque a expressão entre aspas.\n")

# PROMPT QUE INFORMA AO MODELO OS PARAMETROS MAIS BASICOS DAS PERGUNTAS INICIAIS; DATAFRAME, PYTHON, RESULTADO, ETC
pandas_prompt_str = ("Você está trabalhando com um dataframe do pandas em Python chamado `DF`.\n"
                    "{colunas_detalhes}\n\n"
                    "Este é o resultado de `print(DF.head())`:\n"
                    "{df_str}\n\n"
                    "Siga estas instruções:\n"
                    "{instruction_str}\n"
                    "Consulta: {query_str}\n\n"
                    "Expressão:")

# GUIA PARA SINTETIZAR A RESPOSTA ATRAVÉS DA PERGUNTA FEITA
response_synthesis_prompt_str = ("Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
                                "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
                                "Consulta: {query_str}\n\n"
                                "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
                                "Saída do Pandas: {pandas_output}\n\n"
                                "Resposta:"
                                "Ao final, exibir o código usado para gerar a resposta, no formato: O código utilizado foi {pandas_instructions}")

# OBTEM INSTRUÇOES DO PANDAS
pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(instruction_str=instruction_str, colunas_detalhes=descricao_colunas(DF), df_str=DF.head(5))
# EXECUTA AS INSTRUÇÕES
pandas_output_parser = PandasInstructionParser(DF)
# SINTETIZA A RESPOSTA
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

llm = Groq(model='llama3-70b-8192', api_key=groq_key)

In [ ]:
from llama_index.core.query_pipeline import(QueryPipeline as QP, Link, InputComponent)

In [ ]:
qp = QP(modules={'input': InputComponent(),
                 'pandas_prompt': pandas_prompt,
                 'llm1': llm,
                 'pandas_output_parser': pandas_output_parser,
                 'response_synthesis_prompt': response_synthesis_prompt,
                 'llm2': llm}, verbose=True)

# DEFININDO A CADEIA DE EVENTOS DO PIPELINE
qp.add_chain(['input', 'pandas_prompt', 'llm1', 'pandas_output_parser'])

# UNINDO OS PROCESSOS (LINKS)
qp.add_links([Link('input', 'response_synthesis_prompt', dest_key='query_str'),
              Link('llm1', 'response_synthesis_prompt', dest_key='pandas_instructions'),
              Link('pandas_outrput_parser', 'response_synthesis_prompt', dest_key='pandas_output')])

qp.add_link('response_synthesis_prompt', 'llm2')

In [ ]:
response = qp.run(query_str='Qual é a média gasta por cada tipo de cliente?')

In [ ]:
texto = response.message.content
texto_format = textwrap.fill(texto, width=100)
texto_format

### APLICAÇÃO FRONTEND PARA CHAT

In [ ]:
import gradio as gr

def load_data(file_path, DF_ESTADO):
    if (file_path is None) | (file_path == ''):
        return 'Faça o upload do arquivo excel para que seja analisado'
    try:
        DF = pd.read_excel(file_path)
        return 'Arquivo carregado com sucesso!', DF
    except Exception as ERROR:
        return f'Erro ao carregar arquivo: {str(ERROR)}', DF_ESTADO
    
# ============== SERÁ NECESSÁRIO CRIAR AS QUERYS DAS CELULAS ACIMA EM FORMATO DE FUNÇÕES: ===================
# ABAIXO ESTA A CELULA ONDE FOI CONTRUIDA A QUERY EM FORMA DE FUNÇÕES:
def descrição_colunas(df):
    descrição = '\n'.join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
    return "Aqui estão os detalhes das colunas do dataframe:\n" + descrição

def pipeline_consulta(df):
    instruction_str = ("1. Converta a consulta para código Python executável usando Pandas.\n"
                        "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
                        "3. O código deve representar uma solução para a consulta.\n"
                        "4. IMPRIMA APENAS A EXPRESSÃO.\n"
                        "5. Não coloque a expressão entre aspas.\n")

    # PROMPT QUE INFORMA AO MODELO OS PARAMETROS MAIS BASICOS DAS PERGUNTAS INICIAIS; DATAFRAME, PYTHON, RESULTADO, ETC
    pandas_prompt_str = ("Você está trabalhando com um dataframe do pandas em Python chamado `DF`.\n"
                            "{colunas_detalhes}\n\n"
                            "Este é o resultado de `print(DF.head())`:\n"
                            "{df_str}\n\n"
                            "Siga estas instruções:\n"
                            "{instruction_str}\n"
                            "Consulta: {query_str}\n\n"
                            "Expressão:")

    # GUIA PARA SINTETIZAR A RESPOSTA ATRAVÉS DA PERGUNTA FEITA
    response_synthesis_prompt_str = ("Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
                                        "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
                                        "Consulta: {query_str}\n\n"
                                        "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
                                        "Saída do Pandas: {pandas_output}\n\n"
                                        "Resposta:"
                                        "Ao final, exibir o código usado para gerar a resposta, no formato: O código utilizado foi {pandas_instructions}")

    pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(instruction_str=instruction_str, df_str=df.head(5), colunas_detalhes=descrição_colunas(df))

    pandas_output_parser = PandasInstructionParser(df)
    response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)
    
    # Criação do Query Pipeline
    qp = QP(modules={"input": InputComponent(),
                        "pandas_prompt": pandas_prompt,
                        "llm1": llm,
                        "pandas_output_parser": pandas_output_parser,
                        "response_synthesis_prompt": response_synthesis_prompt,
                        "llm2": llm,},verbose=True,)
    
    qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
    qp.add_links([Link("input", "response_synthesis_prompt", dest_key="query_str"),
                  Link("llm1", "response_synthesis_prompt", dest_key="pandas_instructions"),
                  Link("pandas_output_parser", "response_synthesis_prompt", dest_key="pandas_output")])
    qp.add_link("response_synthesis_prompt", "llm2")
    
    return qp

def question_processing(question, DF_ESTADO):
    if not DF_ESTADO.empty:
        qp = pipeline_consulta(DF_ESTADO)
        answer = qp.run(query_str=question)
        return answer.message.content
    else:
        return ''

with gr.Blocks() as app:
    input_file = gr.File(file_count='single', type='filepath', label='Carregar Excel')
    upload_status = gr.Textbox(label='Status do Upload')
    input_question = gr.Textbox(label='Entre sua pergunta')
    button_submit = gr.Button('Enviar')
    output_answer = gr.Textbox(label='Resposta')
    file_pdf = gr.File(label='Download do PDF')

    DF_ESTADO = gr.State(value=None)

    input_file.change(fn=load_data, inputs=[input_file, DF_ESTADO], outputs=[upload_status, DF_ESTADO])
    button_submit.click(fn=question_processing, inputs=[input_question, DF_ESTADO], outputs=output_answer)

app.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
